<a href="https://colab.research.google.com/github/kurota0612/test/blob/main/GitTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("test")

test


In [ ]:
from google.colab import drive
drive.mount('/content/drive')